In [1]:
import torch

print('cuda ',torch.version.cuda,
      '\ndevice ', torch.cuda.get_device_name(0))

cuda  11.6 
device  NVIDIA A100-SXM4-40GB


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install transformers

In [3]:
# import 
from torch.utils.data import Dataset, random_split
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPTNeoForCausalLM


In [4]:
torch.manual_seed(42)
train_file = "training.100000.processed.noemoticon.csv"


In [5]:
seed = 42

# seed
torch.manual_seed(seed)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
model.resize_token_embeddings(len(tokenizer))



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 2048)

In [6]:
#descriptions = pd.read_csv('netflix_titles.csv')['description']

df = pd.read_csv(train_file, encoding='ISO-8859-1', header=None)
df = df[[0, 5]]
df.columns = ['label', 'text']
df = df.sample(30000, random_state=1)

max_length = max([len(tokenizer.encode(description)) for description in df['text']])
print("Max length: {}".format(max_length))

Max length: 156


In [7]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        for txt, label in zip(txt_list, label_list):
            encodings_dict = tokenizer(f'<|startoftext|>Review: {txt}\nSentiment: {map_label[label]}<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [8]:
dataset = NetflixDataset(df['text'].tolist(), df['label'].tolist(), tokenizer, max_length=max_length)
dataset.__getitem__(5)

(tensor([50257, 14832,    25,   318,  2342,   259, 18550,  4813, 16317, 22850,
         18548,  1312,   275,   287,   257,  2008, 16317,  2385,    87,   390,
            88,  8384,   779, 45630,   272,  4813,   220,   198, 31837,  3681,
            25,  4633, 50256, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 5

In [9]:

train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])


In [10]:
print(len(dataset))

30000


In [11]:
save_dir='./result'
training_args = TrainingArguments(output_dir=save_dir, num_train_epochs=5, logging_steps=10000, save_steps=10000,
                                  per_device_train_batch_size=6, per_device_eval_batch_size=6,warmup_steps=50, fp16=True,
                                  weight_decay=0.01, logging_dir=f'./logs1')

In [12]:
trainer=Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})

trainer.train()

Using cuda_amp half precision backend
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27000
  Num Epochs = 5
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 22500
  Number of trainable parameters = 1315579904


Step,Training Loss
10000,0.497400
20000,0.155600


Saving model checkpoint to ./result/checkpoint-10000
Configuration saved in ./result/checkpoint-10000/config.json
Model weights saved in ./result/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./result/checkpoint-20000
Configuration saved in ./result/checkpoint-20000/config.json
Model weights saved in ./result/checkpoint-20000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=22500, training_loss=0.29983243069118926, metrics={'train_runtime': 5706.9234, 'train_samples_per_second': 23.655, 'train_steps_per_second': 3.943, 'total_flos': 1.5270039502848e+17, 'train_loss': 0.29983243069118926, 'epoch': 5.0})

REF : 
https://github.com/SamsTheGreatest/gpt-neo-with-deepspeed/blob/main/gpt%20neo%20with%20deepspeed.ipynb
https://github.com/dredwardhyde/gpt-neo-fine-tuning-example/blob/main/gpt_j_deepspeed.py
https://github.com/Xirider/finetune-gpt2xl/blob/main/ds_config_gptneo.json
https://peter-albert.medium.com/guide-finetune-gpt-neo-2-7-billion-parameters-on-one-gpu-648d049e59b2
https://github.com/Xirider/finetune-gpt2xl

https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e
https://github.com/dredwardhyde/gpt-neo-fine-tuning-example
https://github.com/nomomon/Amazon-Product-Description

In [ ]:
trainer.save_model() 

In [16]:
model.save_pretrained("my_sentiment_model")

Configuration saved in my_imdb_model/config.json
Model weights saved in my_imdb_model/pytorch_model.bin
